In [165]:
import os, os.path
import numpy as np
import pandas as pd
import data_structures as ds
import model_afolu as ma
import model_ippu as mi
import model_circular_economy as mc
import model_energy as me
import model_electricity as ml
import model_socioeconomic as se
from model_socioeconomic import Socioeconomic
import setup_analysis as sa
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
import sqlalchemy
import sqlite3
from typing import Union
# very useful: https://stackoverflow.com/questions/427453/how-can-i-get-the-source-code-of-a-python-function
#print(inspect.getsource(FUNCTIONHERE))
import inspect
import sql_utilities as sqlutil
from julia.api import Julia

importlib.reload(ds)
importlib.reload(sa)
importlib.reload(sf)
importlib.reload(ma)
importlib.reload(mc)
importlib.reload(mi)
importlib.reload(me)
importlib.reload(se)
importlib.reload(ml)


<module 'model_electricity' from '/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/python/model_electricity.py'>

In [2]:
jl = Julia(compiled_modules = False)

In [10]:
from julia import Base
from julia import Pkg
Pkg.activate(sa.dir_jl)

from julia import NemoMod
from julia import Cbc
from julia import JuMP



In [12]:
NemoMod.createnemodb(sa.fp_sqlite_nemomod_db_tmp)

<PyCall.jlwrap SQLite.DB("/Users/jsyme/Documents/Projects/git_jbus/lac_decarbonization/tmp/nemomod_intermediate_database.sqlite")>

In [121]:
engine = sqlalchemy.create_engine(f"sqlite:///{sa.fp_sqlite_nemomod_db_tmp}")

In [23]:
with engine.connect() as con:
    df1 = pd.read_sql_query("select * from OutputActivityRatio;", con)
#df1

In [46]:
sqlutil._write_dataframes_to_db(
    dict_to_sql2,
    engine
)

In [45]:
dict_to_sql2["StorageLevelStart"]

KeyError: 'StorageLevelStart'

<PyCall.jlwrap INFEASIBLE_OR_UNBOUNDED>

In [122]:
##  GENERATE INPUT DICTIONARY

importlib.reload(sf)
importlib.reload(ml)
model_elec = ml.ElectricEnergy(sa.model_attributes)
fp_in = sa.dict_fp_csv_nemomod.get(sa.model_attributes.table_nemomod_specified_demand_profile)
df_spd = pd.read_csv(fp_in)

fp_in = sa.dict_fp_csv_nemomod[sa.model_attributes.table_nemomod_capacity_factor]
df_capfactor = pd.read_csv(fp_in)

#dict_from_storage, dict_to_storage, dict_storage_techs_to_storage = model_elec.format_nemomod_table_technology_from_and_to_storage()
t0 =time.time()
dict_to_sql = model_elec.retrieve_tables_for_sql(df_cs_integrated, df_capfactor, df_spd);


##  REDUCED FOR TESTINS

dict_to_sql2 = {}
#dict_to_sql2["CapacityToActivityUnit"]
write_always = [
    "EMISSION", 
    "FUEL", 
    "MODE_OF_OPERATION",
    "REGION",
    "STORAGE",
    "TECHNOLOGY",
    "YEAR",
    "LTsGroup",
    "TIMESLICE",
    "TSGROUP1",
    "TSGROUP2",
    "YearSplit"
]
others = sorted([
    x for x in dict_to_sql.keys() if x not in write_always
])

for k in write_always + others:
    df_tmp = dict_to_sql[k].copy() 
    if "y" in df_tmp.columns:  
        dict_to_sql2[k] = df_tmp[df_tmp["y"].isin(["2020", "2021"])].reset_index(drop = True)
    elif k == "YEAR":
        dict_to_sql2[k] = df_tmp[df_tmp["val"].isin(["2020", "2021"])].reset_index(drop = True)
    else:
        dict_to_sql2[k] = df_tmp


sqlutil._write_dataframes_to_db(
    dict_to_sql,
    engine
)


In [160]:
NemoMod.calculatescenario(
    sa.fp_sqlite_nemomod_db_tmp, 
    jumpmodel = JuMP.Model(Cbc.Optimizer), 
    numprocs = 1, 
    calcyears = [2020, 2055],
    reportzeros = False,
    varstosave = "vdemandnn, vnewcapacity, vtotalcapacityannual, vannualemissions, vcapitalinvestment, vaccumulatednewstoragecapacity, vproductionbytechnologyannual"
)


<PyCall.jlwrap OPTIMAL>

In [161]:
[x for x in engine.table_names() if x.startswith("v")]

['vaccumulatednewstoragecapacity',
 'vannualemissions',
 'vcapitalinvestment',
 'vdemandnn',
 'vnewcapacity',
 'vproductionbytechnologyannual',
 'vtotalcapacityannual']

In [162]:
with engine.connect() as con:
    df1 = pd.read_sql_query("select * from vproductionbytechnologyannual;", con)
#df1[df1["e"] == "co2"]
df1[df1["f"] == "fuel_electricity"]

,r,t,f,y,val,solvedtm
0,costa_rica,st_flywheels,fuel_electricity,2020,0.025511,2022-08-14 23:49:07.966
3,costa_rica,st_compressed_air,fuel_electricity,2020,0.025511,2022-08-14 23:49:07.966
4,costa_rica,st_batteries,fuel_electricity,2055,0.035925,2022-08-14 23:49:07.966
5,costa_rica,st_compressed_air,fuel_electricity,2055,0.035925,2022-08-14 23:49:07.966
7,costa_rica,pp_geothermal,fuel_electricity,2020,0.005937,2022-08-14 23:49:07.966
8,costa_rica,pp_hydropower,fuel_electricity,2055,0.035925,2022-08-14 23:49:07.966
9,costa_rica,st_batteries,fuel_electricity,2020,0.025511,2022-08-14 23:49:07.966
10,costa_rica,st_flywheels,fuel_electricity,2055,0.035925,2022-08-14 23:49:07.966
13,costa_rica,pp_solar,fuel_electricity,2020,0.019574,2022-08-14 23:49:07.966


In [166]:
importlib.reload(ds)
importlib.reload(mi)

warnings.filterwarnings("ignore")

df_cs_integrated = pd.read_csv(os.path.join(sa.dir_ref, "fake_data", "fake_data_complete.csv"))

model_afolu = ma.AFOLU(sa.model_attributes);
model_circecon = mc.CircularEconomy(sa.model_attributes);
model_ippu = mi.IPPU(sa.model_attributes);
model_energy = me.NonElectricEnergy(sa.model_attributes);

# 
df_project_afolu = model_afolu.project(df_cs_integrated)
df_cs_integrated = sf.merge_output_df_list([df_cs_integrated, df_project_afolu], sa.model_attributes, "concatenate")

# pass integrated df
df_project_ce = model_circecon.project(df_cs_integrated)
df_cs_integrated = sf.merge_output_df_list([df_cs_integrated, df_project_ce], sa.model_attributes, "concatenate")

# pass integrated df
df_project_ip = model_ippu.project(df_cs_integrated)
df_cs_integrated = sf.merge_output_df_list([df_cs_integrated, df_project_ip], sa.model_attributes, "concatenate")

# pass integrated df
df_project_en = model_energy.project(df_cs_integrated)
df_cs_integrated = sf.merge_output_df_list([df_cs_integrated, df_project_en], sa.model_attributes, "concatenate")


In [126]:
julia.install()

,id,r,t,y,val
